In [1]:
import sys
sys.path.extend(['/Users/zsolt/Colloid_git/TractionRheoscopy'])
from data_analysis import static as da
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
from importlib import reload
import numpy as np

In [2]:
hdf_stem = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/locations_stitch/'
sed_fName = 'tfrGel10212018A_shearRun10292018f_sed_stitched.h5'
gel_fName = 'tfrGel10212018A_shearRun10292018f_gel_stitched.h5'

figPath = '/Users/zsolt/Colloid/DATA/tfrGel10212018x/tfrGel10212018A_shearRun10292018f/plots'

In [3]:
sedPos = da.loadData2Mem(hdf_stem + sed_fName)

In [4]:
sed_strainTraj_dt3 = pd.read_hdf(hdf_stem + '/{}'.format('tfrGel10212018A_shearRun10292018f_sed_strainTraj_dt3.h5'))

## Interpolate the strain in 3D using linear nearest neighbor

In [5]:
sed_strainTraj_dt3.head()

(0,1)  (0,3)  (1,4)  (2,5)  (3,6)  (4,7)  (5,8)  (6,9)  \
particle values                                                              
0        D2_min  0.176064    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exx     0.016755    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exy    -0.005082    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         exz     0.009494    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
         eyy    -0.010655    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                 (7,10)  (8,11)  ...  (77,80)  (78,81)  (79,82)  (80,83)  \
particle values                  ...                                       
0        D2_min     NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exx        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exy        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         exz        NaN     NaN  ...      NaN      NaN      NaN      NaN   
         eyy        NaN     NaN  ...      NaN      NaN      NaN      NaN   

                 (81,84)  (82,85)  (83,86)  (84,87)  (85,88)  (86,89)  
particle values                                                        
0        D2_min      NaN      NaN      NaN      NaN      NaN      NaN  
         exx         NaN      NaN      NaN      NaN      NaN      NaN  
         exy         NaN      NaN      NaN      NaN      NaN      NaN  
         exz         NaN      NaN      NaN      NaN      NaN      NaN  
         eyy         NaN      NaN      NaN      NaN      NaN      NaN  

[5 rows x 88 columns]

In [144]:
strain_exz = sed_strainTraj_dt3.loc[(slice(None),'exz'),:]['(0,1)'].unstack().dropna()
idx = strain_exz.index

In [145]:
pts = sedPos.loc[(0,slice(None))][['x (um, imageStack)', 'y (um, imageStack)', 'z (um, imageStack)']].loc[idx]

In [172]:
xx,yy,zz = np.mgrid[10:230:0.7,10:230:0.7,35:70:0.7]

In [147]:
from scipy import interpolate

In [148]:
pts_np = np.array([pts[key].to_numpy() for key in ['x (um, imageStack)', 'y (um, imageStack)', 'z (um, imageStack)']])

In [173]:
strain_interp = interpolate.griddata(np.transpose(pts_np), strain_exz.to_numpy().squeeze(),(xx,yy,zz),method='linear')

In [174]:
strain_interp[np.isnan(strain_interp)].shape 

(0,)

In [175]:
strain_interp[~np.isnan(strain_interp)].shape

(4961250,)

## Write a correlation function with limited shift range

In [181]:
import numba
from data_analysis import test as eshelby
from particleLocating import curvatureFilter

In [ ]:
@numba.jit(nopython=True)
def xcorr(a,b,k):
    """
    Computes the correlation between arrays a and b as a function of shift
    vectors k. 
    """
    out = np.zeros(k)
    for kx in range(-1*k[2], k[2]+1):
        for ky in range(-1*k[1], k[1]+1):
            for kz in range(-1*k[0], k[0]+1):
                # shift b and crop 
                out[kz,ky,kx] = 
                

In [178]:
for n in range(-2,2): print(n)

-2
-1
0
1


## Test of interpolate.griddata

In [101]:
def g(x,y,z): return x*y*z
pts = np.random.rand(10,3)
val = g(pts[:,1],pts[:,1],pts[:,2])

In [103]:
val

array([4.18714336e-01, 3.17938028e-02, 2.04266261e-01, 8.63596029e-07,
       3.73227876e-01, 7.50051477e-04, 5.38846839e-02, 1.95185091e-01,
       1.33393900e-01, 7.22172852e-02])

In [105]:
xx,yy,zz = np.mgrid[0:1:0.1, 0:1:0.1,0:1:0.1]

In [109]:
pts.shape

(10, 3)

In [106]:
interpolate.griddata(pts,val,(xx,yy,zz))

array([[[       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan,        nan,        nan,        nan],
        [       nan,       